# Import Required Libraries

In [45]:
import pandas as pd
from IPython.display import display, HTML
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer 
from nltk.corpus import stopwords
import re

# Merging csv files into one datas

In [46]:
#Loading datasets
devices = pd.read_csv('C:/Users/melin/Documents/Springboard Data Science Career Track/Capstone Projects/Capstone Project 1/icij-imddb-2019-07-09/devices-1562662526.csv')
manufacturers = pd.read_csv('C:/Users/melin/Documents/Springboard Data Science Career Track/Capstone Projects/Capstone Project 1/icij-imddb-2019-07-09/manufacturers-1562662522.csv')
events =  pd.read_csv('C:/Users/melin/Documents/Springboard Data Science Career Track/Capstone Projects/Capstone Project 1/icij-imddb-2019-07-09/events-1562662544.csv')

In [47]:
#Check shapes of devices, manufacturers, and events datasets
devices.shape

(104066, 15)

In [48]:
manufacturers.shape

(26013, 10)

In [49]:
events.shape

(109574, 30)

In [50]:
## merging devices and manufacturers dataframes

#renaming columns in devices dataframe
devices.rename(columns={"id": "device_id"}, inplace=True)
devices.rename(columns={"name": "device_name"}, inplace=True)
devices.rename(columns={"slug": "device_slug"}, inplace=True)
devices.rename(columns={"created_at": "device_created_at"}, inplace=True)
devices.rename(columns={"updated_at": "device_updated_at"}, inplace=True)

#renaming columns in manufacturers dataframe
manufacturers.rename(columns={"id": "manufacturer_id"}, inplace=True)
manufacturers.rename(columns={"name": "manufacturer_name"}, inplace=True)
manufacturers.rename(columns={"slug": "manufacturer_slug"}, inplace=True)
manufacturers.rename(columns={"created_at": "manufacturer_created_at"}, inplace=True)
manufacturers.rename(columns={"updated_at": "manufacturer_updated_at"}, inplace=True)

df = pd.merge(devices,
                 manufacturers,
                 on='manufacturer_id',
                 how='outer')

In [51]:
#Check shape of df dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 104137 entries, 0 to 104136
Data columns (total 24 columns):
device_id                  104066 non-null float64
classification             35179 non-null object
code                       33123 non-null object
description                80627 non-null object
distributed_to             33626 non-null object
implanted                  34898 non-null object
device_name                104061 non-null object
number                     61149 non-null object
quantity_in_commerce       31930 non-null object
risk_class                 32948 non-null object
device_slug                104066 non-null object
country                    104065 non-null object
manufacturer_id            104137 non-null int64
device_created_at          104066 non-null object
device_updated_at          104066 non-null object
address                    47347 non-null object
comment                    12062 non-null object
manufacturer_name          102407 non-null object

In [52]:
## merging df and events dataframes

#renaming columns in events dataframe (indicating which original dataset they came from before the merging of datasets)
df.rename(columns={"number": "device_number"}, inplace=True)
df.rename(columns={"country": "device_country"}, inplace=True)
df.rename(columns={"source": "manufacturer_source"}, inplace=True)

#renaming columns in events dataframe
events.rename(columns={"id": "event_id"}, inplace=True)
events.rename(columns={"country": "event_country"}, inplace=True)
events.rename(columns={"source": "event_source"}, inplace=True)
events.rename(columns={"number": "event_number"}, inplace=True)
events.rename(columns={"slug": "event_slug"}, inplace=True)
events.rename(columns={"created_at": "event_created_at"}, inplace=True)
events.rename(columns={"updated_at": "event_updated_at"}, inplace=True)

df_final = pd.merge(df,
                 events,
                 on='device_id',
                 how='outer')

In [53]:
#Check shape of df_final dataset
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109645 entries, 0 to 109644
Data columns (total 53 columns):
device_id                  109574 non-null float64
classification             37392 non-null object
code                       35278 non-null object
description                84943 non-null object
distributed_to             35782 non-null object
implanted                  37135 non-null object
device_name                109554 non-null object
device_number              63709 non-null object
quantity_in_commerce       33682 non-null object
risk_class                 35091 non-null object
device_slug                109574 non-null object
device_country             109573 non-null object
manufacturer_id            109645 non-null int64
device_created_at          109574 non-null object
device_updated_at          109574 non-null object
address                    50312 non-null object
comment                    12654 non-null object
manufacturer_name          107814 non-null object

# Dealing with Columns with High Missing Percentage

In [54]:
#Remove columns with missing percentage of 60% or more
df_final = df_final.loc[:, df_final.isnull().mean() < .6]

# Manually Remove Redundant Columns

In [55]:
#Remove columns that have no relations to the outcome column, type (Event Type)
df_final = df_final.drop(['description', 'device_number', 'device_created_at', 'device_slug', 'device_updated_at', 'manufacturer_id', 'manufacturer_name', 'address', 'parent_company', 'manufacturer_slug', 'manufacturer_source', 'manufacturer_created_at', 'manufacturer_updated_at', 'action', 'data_notes', 'authorities_link', 'date_initiated_by_firm', 'event_source', 'event_slug', 'icij_notes', 'uid', 'uid_hash', 'url', 'event_created_at', 'event_updated_at'], axis = 1)
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109645 entries, 0 to 109644
Data columns (total 8 columns):
device_id                109574 non-null float64
device_name              109554 non-null object
device_country           109573 non-null object
event_id                 109574 non-null float64
action_classification    52950 non-null object
event_country            109573 non-null object
reason                   61803 non-null object
type                     109573 non-null object
dtypes: float64(2), object(6)
memory usage: 5.0+ MB


# Check and Reorganize Values of Columns

In [56]:
#Checking the value counts of certain columns in df_final
df_final['action_classification'].value_counts() #action_classification column

Class 2                    31282
II                          8216
III                         4705
Class 1                     2483
Class II                    2336
Class 3                     2058
Class I                      907
I                            632
Class III                    327
Unclassified Correction        3
Voluntary recall               1
Name: action_classification, dtype: int64

In [57]:
df_final['type'].value_counts() #type column

Recall                   65242
Field Safety Notice      36182
Recall / Safety Alert     4321
Safety alert              3828
Name: type, dtype: int64

In [58]:
#Changing varied-written values of action_classification into uniform values
df_final[['action_classification']] = df_final[['action_classification']].replace(dict.fromkeys(['I', 'Class I'], 'Class 1'))
df_final[['action_classification']] = df_final[['action_classification']].replace(dict.fromkeys(['II','Class II'], 'Class 2'))
df_final[['action_classification']] = df_final[['action_classification']].replace(dict.fromkeys(['III', 'Class III'], 'Class 3'))

#Removing rows with values of Unclassified Correction and Voluntary recall under the action_classification column (as there are only a handful, their removal will not affect the final analysis of the data)
df_final = df_final[~(df_final["action_classification"].isin(['Unclassified Correction','Voluntary recall']))]

In [59]:
#Remove rows of missing values as type is the outcome variable
df_final = df_final[df_final['type'].notnull()]

In [60]:
#Remove rows containing missing values under the reason column 
df_final = df_final[df_final['reason'].notnull()]
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61799 entries, 956 to 107877
Data columns (total 8 columns):
device_id                61799 non-null float64
device_name              61796 non-null object
device_country           61799 non-null object
event_id                 61799 non-null float64
action_classification    52267 non-null object
event_country            61799 non-null object
reason                   61799 non-null object
type                     61799 non-null object
dtypes: float64(2), object(6)
memory usage: 2.8+ MB


# Text Cleaning

In [61]:
# function for text cleaning 
def clean_text(text): 
    # remove everything except alphabets 
    text = re.sub("[^a-zA-Z]"," ",text) 
    # remove whitespaces 
    text = ' '.join(text.split()) 
    # convert text to lowercase 
    text = text.lower() 
    
    return text

In [62]:
#Apply text cleaning function to reason column in df_final dataframe
df_final['clean_reason'] = df_final['reason'].apply(lambda x: clean_text(x))

In [63]:
# function for stemming words
def stem_text(text):
    ps = PorterStemmer()
    token_words=word_tokenize(str(text))
    token_words
    stem_text=[]
    for word in token_words:
        stem_text.append(ps.stem(word))
        stem_text.append(" ")
    return "".join(stem_text)

In [64]:
#Apply stemming text function to clean_reason column in df_final dataframe
df_final['clean_reason'] = df_final['clean_reason'].apply(lambda x: stem_text(x))

In [65]:
#Remove stop words in clean_reason column

# function to remove stopwords
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    no_stopword_text = [w for w in str(text).split() if not w in stop_words]
    return ' '.join(no_stopword_text)

df_final['clean_reason'] = df_final['clean_reason'].apply(lambda x: remove_stopwords(x))

df_final.head()

,device_id,device_name,device_country,event_id,action_classification,event_country,reason,type,clean_reason
956,861.0,Invacare Action 3 Junior Push Bar Handle,NZL,957.0,NaN,NZL,The failure of the finger bolts can lead to in...,Recall,failur finger bolt lead injuri control chair lost
957,862.0,Bio-Rad D-10 Hemoglobin A Program Reorder pack,NZL,958.0,NaN,NZL,Medical Device Correction iniitiated as a resu...,Recall,medic devic correct iniiti result custom compl...
958,863.0,Bio-Rad VARIANT 11 beta Thalassemmia Short pro...,NZL,959.0,NaN,NZL,Medical Device Correction iniitiated as a resu...,Recall,medic devic correct iniiti result custom compl...
959,1062.0,Bio-Rad IH-Com Kit Fullversion (Data managemen...,NZL,1159.0,NaN,NZL,In case of non interpretable results in the an...,Recall,case non interpret result anti ab well abo id ...
960,1088.0,Bio-Rad Urinary Catecholamines by HPLC Reagent...,NZL,1185.0,NaN,NZL,In some urine samples the concentration of adr...,Recall,urin sampl concentr adrenalin epinephrin may d...


# Export Final Dataset into csv

In [66]:
df_final.to_csv (r'C:/Users/melin/Documents/Springboard Data Science Career Track/Capstone Projects/Capstone Project 1/df_final.csv', index = False, header=True)